In [1]:
import os
import math
from tqdm import tqdm
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.utils import save_image, make_grid
import torchvision.transforms as T

# --- Utilities ---

def exists(x):
    return x is not None

class EMA:
    def __init__(self, model: nn.Module, beta=0.9999):
        self.beta = beta
        self.model = model
        self.shadow = {name: p.clone().detach() for name, p in model.named_parameters() if p.requires_grad}

    def update(self):
        for name, p in self.model.named_parameters():
            if not p.requires_grad:
                continue
            s = self.shadow[name]
            s.sub_((1 - self.beta) * (s - p.detach()))

    def store(self, path: str):
        torch.save(self.shadow, path)

    def copy_to(self, model: nn.Module):
        for name, p in model.named_parameters():
            if name in self.shadow:
                p.data.copy_(self.shadow[name].data)

# --- Noise Schedule (Cosine) ---

def make_cosine_schedule(timesteps: int, s=0.008):
    steps = timesteps + 1
    t = torch.linspace(0, timesteps, steps)
    f = torch.cos(((t / timesteps) + s) / (1 + s) * math.pi * 0.5) ** 2
    alphas_cumprod = f / f[0]
    betas = 1 - (alphas_cumprod[1:] / alphas_cumprod[:-1])
    betas = torch.clamp(betas, 0.0001, 0.999)
    return betas

class DiffusionSchedule:
    def __init__(self, timesteps: int):
        self.timesteps = timesteps
        device = torch.device('cpu')
        betas = make_cosine_schedule(timesteps).to(device)
        alphas = 1. - betas
        alphas_cumprod = torch.cumprod(alphas, dim=0)
        alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)

        self.betas = betas
        self.alphas = alphas
        self.alphas_cumprod = alphas_cumprod
        self.alphas_cumprod_prev = alphas_cumprod_prev
        self.sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
        self.sqrt_one_minus_alphas_cumprod = torch.sqrt(1 - alphas_cumprod)
        self.posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)
        
        # Precompute coefficients for sampling
        self.posterior_mean_coef1 = (betas * torch.sqrt(alphas_cumprod_prev) / (1 - alphas_cumprod))
        self.posterior_mean_coef2 = ((1 - alphas_cumprod_prev) * torch.sqrt(alphas) / (1 - alphas_cumprod))

    def to(self, device):
        for k, v in self.__dict__.items():
            if isinstance(v, torch.Tensor):
                self.__dict__[k] = v.to(device)
        return self

In [3]:
import numpy as np
import os

# --- This cell now inspects your .npy file directly ---

LABELS_NPY_PATH = '/kaggle/input/pixel-art/sprites_labels.npy'
SPRITES_NPY_PATH = '/kaggle/input/pixel-art/sprites.npy'

try:
    print(f"Loading labels from {LABELS_NPY_PATH}...")
    labels = np.load(LABELS_NPY_PATH)
    print(f"Labels loaded. Shape: {labels.shape}")

    # Also load sprites to check for N mismatch
    sprites = np.load(SPRITES_NPY_PATH)
    print(f"Sprites loaded. Shape: {sprites.shape}")

    if labels.shape[0] != sprites.shape[0]:
        print(f"⚠️ Warning: Mismatch in item count!")
        print(f"Sprites: {sprites.shape[0]} items")
        print(f"Labels: {labels.shape[0]} items")

    # --- Auto-detect label format ---
    if labels.ndim == 2:
        # This is One-Hot format, e.g., (N, 5)
        print("Detected One-Hot label format.")
        NUM_CLASSES = labels.shape[1] # e.g., 5
    elif labels.ndim == 1:
        # This is Integer format, e.g., (N,)
        print("Detected Integer label format.")
        NUM_CLASSES = np.max(labels) + 1 # e.g., max is 4, so 5 classes
    else:
        raise ValueError(f"Unexpected label dimension: {labels.ndim}")
        
    print(f"✅ Found {NUM_CLASSES} classes.")
    print(f"The 'null' class index for CFG will be: {NUM_CLASSES}")
    
    print("\n---")
    print(f"➡️ Copy this 'NUM_CLASSES' value ({NUM_CLASSES}) into Cell 6.")

except FileNotFoundError:
    print(f"❌ Error: Could not find {LABELS_NPY_PATH} or {SPRITES_NPY_PATH}")
    print("Please check your file paths.")
except Exception as e:
    print(f"❌ Error inspecting labels: {e}")

Loading labels from /kaggle/input/pixel-art/sprites_labels.npy...
Labels loaded. Shape: (89400, 5)
Sprites loaded. Shape: (89400, 16, 16, 3)
Detected One-Hot label format.
✅ Found 5 classes.
The 'null' class index for CFG will be: 5

---
➡️ Copy this 'NUM_CLASSES' value (5) into Cell 6.


In [4]:
class PixelArtDataset(Dataset):
    def __init__(self, npy_path: str, labels_path: str, image_size=16, augment=False):
        data = np.load(npy_path)
        if data.dtype == np.uint8:
            data = data.astype(np.float32) / 255.0
        
        # data = np.transpose(data, (0, 3, 1, 2)) # <-- DELETE THIS LINE
        
        self.x = data.astype(np.float32) # self.x is now (N, H, W, C)
        
        # --- Updated Label Loading ---
        raw_labels = np.load(labels_path)
        
        if raw_labels.ndim == 2:
            # Convert from One-Hot [1,0,0] to Integer 0
            print("Converting one-hot labels to integers.")
            self.labels = np.argmax(raw_labels, axis=1)
        else:
            # Labels are already Integers
            self.labels = raw_labels
        # ---
        
        self.image_size = image_size
        self.augment = augment
        self.transform = T.Compose([
            T.ToPILImage(), # Now correctly receives (H, W, C)
            T.Resize(image_size, interpolation=Image.NEAREST), # NEAREST for pixel art
            T.ToTensor(),   # This converts the PIL Image to (C, H, W)
        ])

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        # self.x[idx] is now (H, W, C)
        img = (self.x[idx] * 255).astype(np.uint8) 
        
        # This transform pipeline now works perfectly
        img = self.transform(img) 
        
        if self.augment:
            if torch.rand(1) < 0.5:
                img = T.functional.hflip(img)
        
        label = self.labels[idx]
        return img, torch.tensor(label, dtype=torch.long)

In [5]:
#
# --- 🔽🔽🔽 REPLACE YOUR CELL 4 WITH THIS 🔽🔽🔽 ---
#

class SiLU(nn.Module):
    def forward(self, x):
        return x * torch.sigmoid(x)

class ResidualBlock(nn.Module):
    def __init__(self, in_ch, out_ch, time_emb_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.norm1 = nn.GroupNorm(8, out_ch)
        self.norm2 = nn.GroupNorm(8, out_ch)
        self.act = SiLU()
        self.skip = nn.Conv2d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()
        
        # Time and Class embeddings are combined *before* this block
        self.time_mlp = nn.Sequential(nn.Linear(time_emb_dim, out_ch), SiLU())

    def forward(self, x, t_emb):
        h = self.conv1(x)
        h = self.norm1(h)
        h = self.act(h)
        h = h + self.time_mlp(t_emb).view(h.shape[0], -1, 1, 1)
        h = self.conv2(h)
        h = self.norm2(h)
        return self.act(h + self.skip(x))

class AttentionBlock(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.norm = nn.GroupNorm(8, channels)
        self.q = nn.Conv1d(channels, channels, 1)
        self.k = nn.Conv1d(channels, channels, 1)
        self.v = nn.Conv1d(channels, channels, 1)
        self.proj_out = nn.Conv1d(channels, channels, 1)

    def forward(self, x):
        b, c, h, w = x.shape
        x_in = x
        x = self.norm(x)
        x = x.view(b, c, h * w)
        q = self.q(x)
        k = self.k(x)
        v = self.v(x)
        w_ = torch.bmm(q.permute(0, 2, 1), k) * (c ** (-0.5))
        w_ = torch.softmax(w_, dim=-1)
        out = torch.bmm(w_, v.permute(0, 2, 1)).permute(0, 2, 1)
        out = self.proj_out(out)
        out = out.view(b, c, h, w)
        return out + x_in

class ContextUNet(nn.Module):
    def __init__(self, in_ch=3, base_ch=64, channel_mults=(1, 2, 2), num_res_blocks=2, 
                 img_size=16, num_classes=10):
        super().__init__()
        self.in_ch = in_ch
        self.base_ch = base_ch
        self.num_classes = num_classes
        time_emb_dim = base_ch * 4
        
        self.time_mlp = nn.Sequential(
            nn.Linear(1, time_emb_dim), SiLU(), 
            nn.Linear(time_emb_dim, time_emb_dim)
        )
        
        self.label_emb = nn.Embedding(num_classes + 1, time_emb_dim)
        
        chs = [base_ch] + [base_ch * m for m in channel_mults]
        self.init_conv = nn.Conv2d(in_ch, chs[0], 3, padding=1)

        # Down
        self.downs = nn.ModuleList()
        in_channels = chs[0]
        use_attention = (False, True, True) 
        for i in range(len(channel_mults)):
            out_channels = chs[i+1]
            blocks = nn.ModuleList()
            for _ in range(num_res_blocks):
                blocks.append(ResidualBlock(in_channels, out_channels, time_emb_dim))
                in_channels = out_channels
            attn = AttentionBlock(out_channels) if use_attention[i] else nn.Identity()
            self.downs.append(nn.ModuleDict({
                'blocks': blocks,
                'attn': attn,
                'downsample': nn.AvgPool2d(2) if (i < len(channel_mults) - 1) else nn.Identity()
            }))

        # Middle
        self.mid = nn.ModuleDict({
            'block1': ResidualBlock(in_channels, in_channels, time_emb_dim),
            'attn': AttentionBlock(in_channels),
            'block2': ResidualBlock(in_channels, in_channels, time_emb_dim)
        })

        # Up
        self.ups = nn.ModuleList()
        for i in reversed(range(len(channel_mults))):
            out_channels = chs[i]
            skip_channels = chs[i+1]
            
            blocks = nn.ModuleList()
            # --- BUG FIX 2: Corrected 'in_channels' logic for each block ---
            for j in range(num_res_blocks + 1):
                if j == 0:
                    # First block after concat
                    block_in_ch = in_channels + skip_channels
                else:
                    # Subsequent blocks
                    block_in_ch = out_channels
                
                blocks.append(ResidualBlock(block_in_ch, out_channels, time_emb_dim))

            # 'in_channels' for the next up-loop-level is 'out_channels' of this level
            in_channels = out_channels
            # --- End of BUG FIX 2 ---
            
            attn = AttentionBlock(out_channels) if use_attention[i] else nn.Identity()
            
            # --- BUG FIX 1: Corrected 'upsample' logic ---
            # Was: if (i > 0)
            # Should be: if (i < len(channel_mults) - 1) to mirror downsampling
            upsample = nn.Upsample(scale_factor=2, mode='nearest') if (i < len(channel_mults) - 1) else nn.Identity()
            # --- End of BUG FIX 1 ---
            
            self.ups.append(nn.ModuleDict({
                'blocks': blocks,
                'attn': attn,
                'upsample': upsample
            }))

        self.out_conv = nn.Sequential(
            nn.GroupNorm(8, base_ch),
            SiLU(),
            nn.Conv2d(base_ch, in_ch, 3, padding=1)
        )

    def forward(self, x, t_norm, c):
        B = x.shape[0]
        t = t_norm.view(B, 1)
        t_emb = self.time_mlp(t)
        c_emb = self.label_emb(c)
        emb = t_emb + c_emb

        hs = []
        h = self.init_conv(x)
        hs.append(h)
        
        for layer in self.downs:
            for block in layer['blocks']:
                h = block(h, emb)
            h = layer['attn'](h)
            hs.append(h)
            h = layer['downsample'](h)

        h = self.mid['block1'](h, emb)
        h = self.mid['attn'](h)
        h = self.mid['block2'](h, emb)

        for layer in self.ups:
            h = layer['upsample'](h)
            skip = hs.pop()
            h = torch.cat([h, skip], dim=1) # This line should now work
            for block in layer['blocks']:
                h = block(h, emb)
            h = layer['attn'](h)

        out = self.out_conv(h)
        return out

#
# --- 🔼🔼🔼 REPLACE YOUR CELL 4 WITH THIS 🔼🔼🔼 ---
#

In [6]:
@torch.no_grad()
def sample_loop(model: ContextUNet, schedule: DiffusionSchedule, img_size: int, n_samples: int, 
                cond_label: int, null_class_idx: int, guidance_scale: float, device='cuda'):
    
    model.eval()
    T = schedule.timesteps
    shape = (n_samples, 3, img_size, img_size)
    x = torch.randn(shape, device=device)
    
    # The class label you want to generate
    c = torch.full((n_samples,), cond_label, dtype=torch.long, device=device)
    # The unconditional "null" label
    c_uncond = torch.full((n_samples,), null_class_idx, dtype=torch.long, device=device)

    for i in tqdm(range(T - 1, -1, -1), desc=f"Sampling (w={guidance_scale})", leave=False):
        t_norm = torch.full((n_samples,), i / T, device=device)
        
        # 1. Get conditional prediction (with label)
        eps_cond = model(x, t_norm, c)
        
        # 2. Get unconditional prediction (with "null" label)
        eps_uncond = model(x, t_norm, c_uncond)
        
        # 3. Combine them using the guidance scale
        eps = eps_uncond + guidance_scale * (eps_cond - eps_uncond)
        
        # --- DDIM-like sampling step ---
        sqrt_alphas_cumprod = schedule.sqrt_alphas_cumprod[i].to(device)
        sqrt_one_minus = schedule.sqrt_one_minus_alphas_cumprod[i].to(device)
        
        # Predict x0
        x0_pred = (x - sqrt_one_minus * eps) / sqrt_alphas_cumprod
        x0_pred = x0_pred.clamp(-1., 1.)
        
        # Get mean and variance
        posterior_mean = (
            schedule.posterior_mean_coef1[i].to(device) * x0_pred +
            schedule.posterior_mean_coef2[i].to(device) * x
        )
        
        if i > 0:
            noise = torch.randn_like(x)
            var = schedule.posterior_variance[i].to(device)
            x = posterior_mean + torch.sqrt(var) * noise
        else:
            x = posterior_mean # No noise at final step
            
    model.train() # Set back to train mode
    return x.clamp(-1, 1)

In [7]:
# ----------------- Hyperparameters -----------------
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# --- 🔽🔽🔽 EDIT THIS 🔽🔽🔽 ---
# Get this value from running Cell 2
YOUR_NUM_CLASSES = 5 # Example value, change this!
# --- 🔼🔼🔼 EDIT THIS 🔼🔼🔼 ---

timesteps = 1000
batch_size = 128
img_size = 16 # Keep this low for pixel art
lr = 2e-4
n_epochs = 100 # Increase this for better results (e.g., 100-200)
base_ch = 64
p_uncond = 0.1 # Probability of dropping label for CFG (10%)
save_dir = '/kaggle/working/pixel_diffusion_outputs'
os.makedirs(save_dir, exist_ok=True)

# The "null" class ID is just the number of classes
# (e.g., if you have 50 classes 0-49, the null ID is 50)
NULL_CLASS_IDX = YOUR_NUM_CLASSES

# dataset paths (Kaggle)
data_npy = '/kaggle/input/pixel-art/sprites.npy'
labels_npy = '/kaggle/input/pixel-art/sprites_labels.npy'

# 1. Load Data
try:
    dataset = PixelArtDataset(data_npy, labels_npy, image_size=img_size, augment=True)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True)
    print(f"Loaded dataset with {len(dataset)} images.")
except Exception as e:
    print(f"❌ Error loading dataset: {e}")
    print("Please check file paths and ensure Cell 3 is correct.")
    # Stop execution if data fails
    raise e

# 2. Build Schedule
schedule = DiffusionSchedule(timesteps).to(device)

# 3. Build Model
# Pass the correct num_classes
model = ContextUNet(
    in_ch=3, 
    base_ch=base_ch, 
    channel_mults=(1, 2, 2), 
    img_size=img_size, 
    num_classes=YOUR_NUM_CLASSES
).to(device)
ema = EMA(model, beta=0.9995)

print(f"Model parameters: {sum(p.numel() for p in model.parameters())/1e6:.2f}M")

optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)

# This line is correct
scaler = torch.amp.GradScaler(device='cuda')

# 4. Training Loop
global_step = 0
for epoch in range(n_epochs):
    model.train()
    pbar = tqdm(dataloader)
    pbar.set_description(f"Epoch {epoch}/{n_epochs}")
    
    for x, c in pbar:
        x = x.to(device) * 2 - 1 # Normalize to [-1, 1]
        c = c.to(device)
        b = x.shape[0]

        # --- Classifier-Free Guidance Training ---
        uncond_mask = (torch.rand(b, device=device) < p_uncond)
        c[uncond_mask] = NULL_CLASS_IDX
        # ---

        t_idx = torch.randint(0, timesteps, (b,), device=device)
        noise = torch.randn_like(x)
        
        sqrt_alpha_cumprod = schedule.sqrt_alphas_cumprod[t_idx].view(-1, 1, 1, 1)
        sqrt_one_minus = schedule.sqrt_one_minus_alphas_cumprod[t_idx].view(-1, 1, 1, 1)
        x_t = sqrt_alpha_cumprod * x + sqrt_one_minus * noise
        
        t_norm = t_idx.float() / timesteps # Model expects t in [0,1]

        optimizer.zero_grad()
        
        # --- CORRECTED autocast (positional argument) ---
        with torch.amp.autocast('cuda', enabled=(device.type == 'cuda')):
            pred = model(x_t, t_norm, c) # Pass the (potentially dropped) label
            loss = F.mse_loss(pred, noise)

        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()

        ema.update()
        global_step += 1

        if global_step % 200 == 0:
            pbar.set_description(f"Epoch {epoch} | loss: {loss.item():.4f}")

    # --- End of Epoch: Validation Sampling ---
    if epoch % 5 == 0 or epoch == n_epochs - 1:
        ema.copy_to(model)
        model.eval()
        
        # Sample 8 different classes (or fewer if you don't have 8)
        n_sample_classes = min(YOUR_NUM_CLASSES, 8)
        samples_per_class = 8
        
        all_samples = []
        for class_id in range(n_sample_classes):
            print(f"Sampling class {class_id}...")
            samples = sample_loop(
                model=model,
                schedule=schedule,
                img_size=img_size,
                n_samples=samples_per_class,
                cond_label=class_id,
                null_class_idx=NULL_CLASS_IDX,
                guidance_scale=7.5, # Standard guidance scale
                device=device
            )
            all_samples.append(samples)
        
        grid_samples = torch.cat(all_samples)
        grid_samples = (grid_samples + 1) / 2 # Denormalize to [0,1]
        
        grid = make_grid(grid_samples, nrow=samples_per_class)
        save_image(grid, os.path.join(save_dir, f'samples_epoch_{epoch}.png'))
        print(f'\n✅ Saved sample grid for epoch {epoch} to {save_dir}')

# --- Final Save ---
ema.store(os.path.join(save_dir, 'ema_shadow.pth'))
print('✅ Training complete. Final EMA weights saved to ema_shadow.pth')

Using device: cuda
Converting one-hot labels to integers.
Loaded dataset with 89400 images.
Model parameters: 4.32M


Epoch 0 | loss: 0.1639: 100%|██████████| 699/699 [00:56<00:00, 12.41it/s]


Sampling class 0...


Sampling class 1...


Sampling class 2...


...

Sampling class 1...


Sampling class 2...


Sampling class 3...


Sampling class 4...



✅ Saved sample grid for epoch 99 to /kaggle/working/pixel_diffusion_outputs
✅ Training complete. Final EMA weights saved to ema_shadow.pth
